# Import libraries

In [1]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset, TensorDataset
import optuna, datetime

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule
# from pytorch_lightning.loggers import WandbLogger
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
import cv2
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import shutil
import pickle
from torch.utils.data.sampler import WeightedRandomSampler
import time
import gc
warnings.filterwarnings("ignore")

# Meta info

In [2]:
drive_root = '..'
TRAIN_DIR = f"{drive_root}/input/petfinder-pawpularity-score/train"
TEST_DIR = f"{drive_root}/input/petfinder-pawpularity-score/test"
DENSE_FEATURES = [
    'Subject Focus',
    'Eyes',
    'Face',
    'Near',
    'Action',
    'Accessory',
    'Group',
    'Collage',
    'Human',
    'Occlusion',
    'Info',
    'Blur',
    'img_long_axis'
]
df_train = pd.read_csv(f"{drive_root}/input/petfinder-pawpularity-score/train.csv")
df_test = pd.read_csv(f"{drive_root}/input/petfinder-pawpularity-score/test.csv")
df_train['filepath'] = df_train.Id.apply(lambda x :f"{TRAIN_DIR}/{x}.jpg" )
df_test['filepath'] = df_test.Id.apply(lambda x :f"{TEST_DIR}/{x}.jpg" )

long_axis_df_path = f"{drive_root}/input/pet-train-long-axis/df_train_w_long_axis.csv"
long_axis_max = 1280
if os.path.isfile(long_axis_df_path):
    df_train = pd.read_csv(long_axis_df_path)
else:
    df_train['img_long_axis'] = df_train.Id.apply(lambda x : max(cv2.imread(f"{TRAIN_DIR}/{x}.jpg" ).shape[:2]))
    df_train['img_long_axis'] /=long_axis_max
    df_train['img_long_axis'] = df_train['img_long_axis'].astype(np.float32)
    df_train['filepath'] = df_train.Id.apply(lambda x :f"{TRAIN_DIR}/{x}.jpg" )
    
df_test['img_long_axis'] = df_test.Id.apply(lambda x : max(cv2.imread(f"{TEST_DIR}/{x}.jpg" ).shape[:2]))
df_test['img_long_axis'] /=long_axis_max
df_test['img_long_axis'] = df_test['img_long_axis'].astype(np.float32)
df_test['filepath'] = df_test.Id.apply(lambda x :f"{TEST_DIR}/{x}.jpg" )

# Losses

In [3]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

class BayesianLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        
    def forward(self,yhat,y,std):
        y = y/100.
        ce = self.bce(yhat, y)
        inv_std = torch.exp(-std)
        mce = inv_std * ce
        loss = 0.5 * (mce + std).mean()
        return loss


# Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, x, y=None):#, image_size=224):
        self._X = x
        self._y = y

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        feature = self._X[idx]
        if self._y is not None:
            label = self._y[idx]
            return feature, label
        return feature
    
class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        train_features, train_y, val_features, val_y, cfg,):
        super().__init__()
        self._train_features = train_features
        self._val_features = val_features
        self._train_y = train_y
        self._val_y = val_y
        self._cfg = cfg

    def __create_dataset(self, train=True):
        if train==True:
            return CustomDataset(self._train_features, self._train_y)
        else:
            return CustomDataset(self._val_features, self._val_y)

    def train_dataloader(self):
        dataset = self.__create_dataset(True)
        return DataLoader(dataset, **self._cfg.train_loader)

    def val_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset, **self._cfg.val_loader)

# EarlyStopper

In [5]:
class EarlyStopper():
    def __init__(self, patience: int, mode:str)-> None:
        self.patience = patience
        self.mode = mode

        # Initiate
        self.patience_counter = 0
        self.stop = False
        self.best_loss = np.inf
        
    def check_early_stopping(self, loss: float)-> None:
        loss = -loss if self.mode == 'max' else loss  # get max value if mode set to max

        if loss >= self.best_loss:
            # got better score
            self.patience_counter += 1
            
            if self.patience_counter == self.patience:
                self.stop = True  # end

        elif loss < self.best_loss:
            # got worse score
            self.patience_counter = 0
            self.best_loss = loss
            


# Configuration

In [6]:
config = {'seed': 2021,
          'root': f"{drive_root}", 
          'n_splits': 10,
          'epoch': 1000,#30,
          'patience':10,
          'train_loader':{
              'batch_size': 512,
              'shuffle': True,
              'num_workers': 1,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': 512,#16,#64,
              'shuffle': False,
              'num_workers': 8,
              'pin_memory': False,
              'drop_last': False
         },
          'save_discript':'Sigmoid_Bayesian',
          'model':{
              'name': 'swin_large_patch4_window12_384_in22k',#'deit_base_distilled_patch16_384',#'swin_tiny_patch4_window7_224',#
              'h1':128,
              'h2':128,
              'output_dim': 1,
              'first_drop': 0.5,
              'second_drop': 0.5,
              'third_drop': 0.5,
              'last_drop':0.5,
              'activation':'nn.SELU',
              'quantile_num':512,
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'AdamW':{
                  'lr': 1e-5,
                  'betas': (1,1),
                  'weight_decay': 0.01,
                  'amsgrad': False,
                  
                },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'CosineAnnealingWarmRestarts':{
                  'T_0': 4,
                  'T_mult':2,
                  'eta_min': 1e-7,
              }
          },
          'metric': 'RMSELoss',
}

config = Box(config)

# Setting arguments

In [7]:
torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
model_code = 'swin_2021'
import random 
torch.manual_seed(config.seed)
torch.cuda.manual_seed(config.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(config.seed)
random.seed(config.seed)

skf = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)
criterion = BayesianLoss()
activation_list = ['nn.LogSigmoid', 'nn.Hardswish',  'nn.PReLU', 'nn.ReLU', 'nn.ELU', 'nn.ReLU6', 'nn.SELU', 'nn.CELU']

Global seed set to 2021


# Base performance

In [8]:
swin_scores = []
model_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}/'
for fold, (train_idx, val_idx) in enumerate(skf.split(df_train["Id"], df_train["Pawpularity"])):
    # Split k-fold    
    train_df = df_train.loc[train_idx].reset_index(drop=True)
    val_df = df_train.loc[val_idx].reset_index(drop=True)
    
    # Load embed features
    val_predicts = np.load(f'{model_save_dir}val_predicts_fold{fold}.npy')
    swin_score = mean_squared_error(val_df.Pawpularity, val_predicts)**0.5
    swin_scores.append(swin_score)
    print(f'fold-{fold} score : {swin_score}')
cv_score = sum(swin_scores) / len(swin_scores)
print(f'cv score : {cv_score}')

fold-0 score : 17.638196173600214
fold-1 score : 17.280833894034252
fold-2 score : 17.21898402640529
fold-3 score : 17.22565662554061
fold-4 score : 17.117359526160584
fold-5 score : 17.340641942008737
fold-6 score : 17.64576757996476
fold-7 score : 17.82393560768342
fold-8 score : 18.049144425526592
fold-9 score : 16.8705192618633
cv score : 17.42110390627878


# Define objective

In [9]:
def objective(trial):
    #Set hyperparams
    config.optimizer.AdamW.lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    config.model.h1 = trial.suggest_categorical('h1', [32, 64, 128, 256, 512, 1024, 2048])
    config.model.h2 = trial.suggest_categorical('h2', [32, 64, 128, 256, 512, 1024, 2048])

    config.patience =  trial.suggest_categorical('patience', [20, 30, 50, 100])
    config.model.activation = trial.suggest_categorical('activation', activation_list)
    config.model.first_drop = trial.suggest_float('first_drop', 0, 0.9)
    config.model.second_drop = trial.suggest_float('second_drop', 0, 0.9)
    config.model.third_drop = trial.suggest_float('third_drop', 0, 0.9)
    
    beta1 = trial.suggest_float('beta1', 0, 1)
    beta2 = trial.suggest_float('beta2', 0, 1)
    config.optimizer.AdamW.betas = (beta1, beta2)
    config.optimizer.AdamW.weight_decay = trial.suggest_float('weight_decay', 0, 1)
    config.optimizer.AdamW.amsgrad = trial.suggest_categorical('amsgrad', [True, False])
    config.scheduler.CosineAnnealingWarmRestarts.T_0 = trial.suggest_int('T_0', 1, 100)
    config.scheduler.CosineAnnealingWarmRestarts.T_mult = trial.suggest_int('T_mult', 1, 100)
    config.scheduler.CosineAnnealingWarmRestarts.eta_min = trial.suggest_loguniform('eta_min', 1e-10, 1e-5)
    model_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}/'

    swin_scores=[]
    val_scores=[]
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df_train["Id"], df_train["Pawpularity"])):
        
        early_stopper = EarlyStopper(patience = config.patience, mode='min')
        val_min_score = 1e+100
        
        train_df = df_train.loc[train_idx].reset_index(drop=True)
        val_df = df_train.loc[val_idx].reset_index(drop=True)

        # Load embed features
        val_predicts = np.load(f'{model_save_dir}val_predicts_fold{fold}.npy')
        train_embed_features = np.load(f'{model_save_dir}train_embed_org_fold{fold}.npy')
        val_embed_features = np.load(f'{model_save_dir}val_embed_org_fold{fold}.npy')
        train_embed_flip_features = np.load(f'{model_save_dir}train_embed_flip_fold{fold}.npy')
        val_embed_flip_features = np.load(f'{model_save_dir}val_embed_flip_fold{fold}.npy')
        train_embed_merge_features = np.load(f'{model_save_dir}train_embed_merge_org_fold{fold}.npy')
        val_embed_merge_features = np.load(f'{model_save_dir}val_embed_merge_org_fold{fold}.npy')
        train_embed_merge_flip_features = np.load(f'{model_save_dir}train_embed_merge_flip_fold{fold}.npy')
        val_embed_merge_flip_features = np.load(f'{model_save_dir}val_embed_merge_flip_fold{fold}.npy')
        swin_score = mean_squared_error(val_df.Pawpularity,val_predicts)**0.5
        train_concat_features = np.concatenate((train_embed_merge_features, train_embed_merge_flip_features), axis=0)
            
        train_loader = DataLoader(CustomDataset(train_concat_features, pd.concat([train_df['Pawpularity'], train_df['Pawpularity']]).reset_index(drop=True)), shuffle=config.train_loader.shuffle, batch_size=config.train_loader.batch_size)

        # Model
        q = nn.Sequential(
                nn.Dropout(config.model.first_drop), 
                nn.Linear(train_concat_features.shape[1], config.model.h1),
                eval(config.model.activation)(),
                nn.Dropout(config.model.second_drop), 
                nn.Linear(config.model.h1, config.model.h2),
                eval(config.model.activation)(),
                nn.Dropout(config.model.third_drop), 
                nn.Linear(config.model.h2, 2)   
        ).to('cuda')
        
        # Optimizer & scheduler
        optimizer = eval(config.optimizer.name)(q.parameters(), **config.optimizer.AdamW)
        scheduler = eval(config.scheduler.name)(optimizer, **config.scheduler.CosineAnnealingWarmRestarts)

        # Train
        for epoch in range(config.epoch):
            
            q.train()
            train_loss = 0
            
            for feature, label in train_loader:
                
                optimizer.zero_grad()
                
                feature, label = feature.to('cuda') , label.to('cuda')
                
                output = q(feature)
                
                pred = output[:, 0]
                
                std = output[:, 1]
                
                loss = criterion(pred.float(), label.float(), std.float())
                
                try:
                    loss.backward()
                    
                    optimizer.step()
                    
                    scheduler.step()
                except:
                    break
                
                train_loss += loss.item()
                
            q.eval()
            pred_lst = []
            val_loss = 0
            with torch.no_grad():
                org_feature = torch.FloatTensor(val_embed_merge_features).to('cuda')
                flip_feature = torch.FloatTensor(val_embed_merge_flip_features).to('cuda')
                org_pred = q(org_feature)
                flip_pred = q(flip_feature)
                org_pred = torch.sigmoid(org_pred[:, 0]).cpu().numpy() * 100    
                flip_pred = torch.sigmoid(flip_pred[:, 0]).cpu().numpy() * 100    
                pred = (org_pred + flip_pred)/2.
            try:    
                val_score = mean_squared_error(val_df.Pawpularity,pred)**0.5
            except:
                val_min_score = 1e+100
                break
            #wandb.log({f'fold-{fold}-score' : val_score})
                
            early_stopper.check_early_stopping(loss=val_score)

            if early_stopper.patience_counter == 0:
                #print(pred_lst[:10])    
                val_min_score = val_score
                print(f'fold {fold} epoch {epoch} swin_score : {swin_score:.2f} val score : {val_score:.2f}', end = '\r')

            if early_stopper.stop == True:
                break
                
        swin_scores.append(swin_score)
        val_scores.append(val_min_score)
        print(f'fold {fold} epoch {epoch} swin_score : {swin_score:.2f} val score : {val_min_score:.2f}')
        trial.report(val_min_score, fold)
            
        if trial.should_prune():
            raise optuna.TrialPruned()
            
    cv_score = sum(val_scores)/len(val_scores)
    return cv_score

# Study Object

In [10]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=2)
study_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}_2nd_head/by_head/'

with open(f"{study_save_dir}03-bayes-head.pkl", 'wb') as file:
    pickle.dump(study, file)

trial = study.best_trial
print(f"Best trial:{trial.number}")

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-01-04 04:30:02,572] A new study created in memory with name: no-name-d28d05fa-1591-47cf-836e-cbb4226fa6f0


fold 0 epoch 35 swin_score : 17.64 val score : 17.61
fold 1 epoch 37 swin_score : 17.28 val score : 17.36
fold 2 epoch 43 swin_score : 17.22 val score : 17.22
fold 3 epoch 33 swin_score : 17.23 val score : 17.18
fold 4 epoch 50 swin_score : 17.12 val score : 17.08
fold 5 epoch 32 swin_score : 17.34 val score : 17.28
fold 6 epoch 32 swin_score : 17.65 val score : 17.71
fold 7 epoch 78 swin_score : 17.82 val score : 17.69
fold 8 epoch 32 swin_score : 18.05 val score : 17.98


[I 2022-01-04 04:34:46,170] Trial 0 finished with value: 17.40202666870052 and parameters: {'learning_rate': 4.03101451487439e-05, 'h1': 128, 'h2': 2048, 'patience': 30, 'activation': 'nn.ELU', 'first_drop': 0.18422146877700166, 'second_drop': 0.8726856982406839, 'third_drop': 0.1884874062528294, 'beta1': 0.5943550982395492, 'beta2': 0.7395579948647513, 'weight_decay': 0.5118359809251878, 'amsgrad': False, 'T_0': 22, 'T_mult': 20, 'eta_min': 5.741467780216835e-09}. Best is trial 0 with value: 17.40202666870052.


fold 9 epoch 47 swin_score : 16.87 val score : 16.91
fold 0 epoch 226 swin_score : 17.64 val score : 17.57
fold 1 epoch 122 swin_score : 17.28 val score : 17.25
fold 2 epoch 213 swin_score : 17.22 val score : 17.24
fold 3 epoch 201 swin_score : 17.23 val score : 17.14
fold 4 epoch 251 swin_score : 17.12 val score : 17.14
fold 5 epoch 106 swin_score : 17.34 val score : 17.26
fold 6 epoch 104 swin_score : 17.65 val score : 17.67
fold 7 epoch 213 swin_score : 17.82 val score : 17.80
fold 8 epoch 117 swin_score : 18.05 val score : 17.86


[I 2022-01-04 04:54:33,470] Trial 1 finished with value: 17.380463219790755 and parameters: {'learning_rate': 0.00012943554116751053, 'h1': 512, 'h2': 64, 'patience': 100, 'activation': 'nn.ReLU', 'first_drop': 0.37206551532758525, 'second_drop': 0.6041437247265542, 'third_drop': 0.6929849228298111, 'beta1': 0.6592273323867845, 'beta2': 0.2596641022558449, 'weight_decay': 0.08058311101056415, 'amsgrad': True, 'T_0': 50, 'T_mult': 57, 'eta_min': 1.1344056554704253e-08}. Best is trial 1 with value: 17.380463219790755.


fold 9 epoch 198 swin_score : 16.87 val score : 16.86
Best trial:1
  Params: 
    learning_rate: 0.00012943554116751053
    h1: 512
    h2: 64
    patience: 100
    activation: nn.ReLU
    first_drop: 0.37206551532758525
    second_drop: 0.6041437247265542
    third_drop: 0.6929849228298111
    beta1: 0.6592273323867845
    beta2: 0.2596641022558449
    weight_decay: 0.08058311101056415
    amsgrad: True
    T_0: 50
    T_mult: 57
    eta_min: 1.1344056554704253e-08


# Train & Save Best Model 

In [11]:
study_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}_2nd_head/by_head/'
model_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}/'

with open(f"{study_save_dir}03-bayes-head.pkl", 'rb') as file:
    study = pickle.load(file)
trial = study.best_trial

In [13]:
#Set hyperparams
config.optimizer.AdamW.lr = trial.params['learning_rate']
config.patience =  trial.params['patience']
config.model.activation = trial.params['activation']
config.model.first_drop = trial.params['first_drop']
config.model.second_drop = trial.params['second_drop']
config.model.third_drop = trial.params['third_drop']
config.model.h1 = trial.params['h1']
config.model.h2 = trial.params['h2']

beta1 = trial.params['beta1']
beta2 = trial.params['beta2']
config.optimizer.AdamW.betas = (beta1, beta2)
config.optimizer.AdamW.weight_decay = trial.params['weight_decay']
config.optimizer.AdamW.amsgrad = trial.params['amsgrad']
config.scheduler.CosineAnnealingWarmRestarts.T_0 = trial.params['T_0']
config.scheduler.CosineAnnealingWarmRestarts.T_mult = trial.params['T_mult']
config.scheduler.CosineAnnealingWarmRestarts.eta_min = trial.params['eta_min']

swin_scores=[]
val_scores=[]
for fold, (train_idx, val_idx) in enumerate(skf.split(df_train["Id"], df_train["Pawpularity"])):

    early_stopper = EarlyStopper(patience = config.patience, mode='min')
    val_min_score = 1e+100
    head_dir = f'{drive_root}/output/weights/final_weights/{model_code}_2nd_head/by_head/BayesHead_fold{fold}.ckpt'

    train_df = df_train.loc[train_idx].reset_index(drop=True)
    val_df = df_train.loc[val_idx].reset_index(drop=True)

    # Load embed features
    val_predicts = np.load(f'{model_save_dir}val_predicts_fold{fold}.npy')
    train_embed_features = np.load(f'{model_save_dir}train_embed_org_fold{fold}.npy')
    val_embed_features = np.load(f'{model_save_dir}val_embed_org_fold{fold}.npy')
    train_embed_flip_features = np.load(f'{model_save_dir}train_embed_flip_fold{fold}.npy')
    val_embed_flip_features = np.load(f'{model_save_dir}val_embed_flip_fold{fold}.npy')
    train_embed_merge_features = np.load(f'{model_save_dir}train_embed_merge_org_fold{fold}.npy')
    val_embed_merge_features = np.load(f'{model_save_dir}val_embed_merge_org_fold{fold}.npy')
    train_embed_merge_flip_features = np.load(f'{model_save_dir}train_embed_merge_flip_fold{fold}.npy')
    val_embed_merge_flip_features = np.load(f'{model_save_dir}val_embed_merge_flip_fold{fold}.npy')

   # DataLoader
    train_concat_features = np.concatenate((train_embed_merge_features, train_embed_merge_flip_features), axis=0)
    train_loader = DataLoader(CustomDataset(train_concat_features, pd.concat([train_df['Pawpularity'], train_df['Pawpularity']]).reset_index(drop=True)), shuffle=config.train_loader.shuffle, batch_size=config.train_loader.batch_size)

   # Model
    q = nn.Sequential(
            nn.Dropout(config.model.first_drop), 
            nn.Linear(train_concat_features.shape[1], config.model.h1),
            eval(config.model.activation)(),
            nn.Dropout(config.model.second_drop), 
            nn.Linear(config.model.h1, config.model.h2),
            eval(config.model.activation)(),
            nn.Dropout(config.model.third_drop), 
            nn.Linear(config.model.h2, 2)   
    ).to('cuda')

    # Optimizer & scheduler
    optimizer = eval(config.optimizer.name)(q.parameters(), **config.optimizer.AdamW)
    scheduler = eval(config.scheduler.name)(optimizer, **config.scheduler.CosineAnnealingWarmRestarts)

    # Train
    for epoch in range(config.epoch):

        q.train()
        train_loss = 0

        for feature, label in train_loader:

            optimizer.zero_grad()

            feature, label = feature.to('cuda') , label.to('cuda')

            output = q(feature)

            pred = output[:, 0]

            std = output[:, 1]

            loss = criterion(pred.float(), label.float(), std.float())

            try:
                loss.backward()

                optimizer.step()

                scheduler.step()
            except:
                break

            train_loss += loss.item()

        q.eval()
        pred_lst = []
        val_loss = 0
        with torch.no_grad():
            org_feature = torch.FloatTensor(val_embed_merge_features).to('cuda')
            flip_feature = torch.FloatTensor(val_embed_merge_flip_features).to('cuda')
            org_pred = q(org_feature)
            flip_pred = q(flip_feature)
            org_pred = torch.sigmoid(org_pred[:, 0]).cpu().numpy() * 100    
            flip_pred = torch.sigmoid(flip_pred[:, 0]).cpu().numpy() * 100    
            pred = (org_pred + flip_pred)/2.

        try:    
            val_score = mean_squared_error(val_df.Pawpularity,pred)**0.5
        except:
            val_min_score = 1e+100
            break
        #wandb.log({f'fold-{fold}-score' : val_score})

        swin_score = mean_squared_error(val_df.Pawpularity,val_predicts)**0.5
        swin_scores.append(swin_score)


        early_stopper.check_early_stopping(loss=val_score)

        if early_stopper.patience_counter == 0:
            pickle.dump(q, open(head_dir, "wb"))
            val_min_score = val_score
            print(f'fold {fold} epoch {epoch} swin_score : {swin_score:.2f} val score : {val_score:.2f}', end = '\r')

        if early_stopper.stop == True:
            break

    val_scores.append(val_min_score)
    print(f'fold {fold} epoch {epoch} swin_score : {swin_score:.2f} val score : {val_min_score:.2f}')

cv_score = sum(val_scores)/len(val_scores)
print(f'cv score : {cv_score}')


fold 0 epoch 192 swin_score : 17.64 val score : 17.58
fold 1 epoch 123 swin_score : 17.28 val score : 17.28
fold 2 epoch 205 swin_score : 17.22 val score : 17.21
fold 3 epoch 122 swin_score : 17.23 val score : 17.15
fold 4 epoch 258 swin_score : 17.12 val score : 17.14
fold 5 epoch 103 swin_score : 17.34 val score : 17.30
fold 6 epoch 126 swin_score : 17.65 val score : 17.76
fold 7 epoch 228 swin_score : 17.82 val score : 17.64
fold 8 epoch 208 swin_score : 18.05 val score : 17.91
fold 9 epoch 187 swin_score : 16.87 val score : 16.92
cv score : 17.387258526233836


# Make prediction file

In [15]:
for fold, (train_idx, val_idx) in enumerate(skf.split(df_train["Id"], df_train["Pawpularity"])):
    model_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}/'
    head_dir = f'{drive_root}/output/weights/final_weights/{model_code}_2nd_head/by_head/BayesHead_fold{fold}.ckpt'
    result_save_dir = f'{drive_root}/output/weights/final_weights/{model_code}_2nd_head/by_head/'
    
    train_df = df_train.loc[train_idx].reset_index(drop=True)
    val_df = df_train.loc[val_idx].reset_index(drop=True)
    
    # Load embed features
    val_predicts = np.load(f'{model_save_dir}val_predicts_fold{fold}.npy')
    val_embed_features = np.load(f'{model_save_dir}val_embed_org_fold{fold}.npy')
    val_embed_flip_features = np.load(f'{model_save_dir}val_embed_flip_fold{fold}.npy')
    val_embed_merge_features = np.load(f'{model_save_dir}val_embed_merge_org_fold{fold}.npy')
    val_embed_merge_flip_features = np.load(f'{model_save_dir}val_embed_merge_flip_fold{fold}.npy')
    
    org_feature = torch.FloatTensor(val_embed_merge_features).to('cuda')
    flip_feature = torch.FloatTensor(val_embed_merge_flip_features).to('cuda')
    q = pickle.load(open(head_dir, "rb"))
    q.eval()   
    with torch.no_grad():
        org_pred = q(org_feature)
        flip_pred = q(flip_feature)
        org_pred = torch.sigmoid(org_pred[:, 0]).cpu().numpy() * 100    
        flip_pred = torch.sigmoid(flip_pred[:, 0]).cpu().numpy() * 100    
        pred = (org_pred + flip_pred)/2.
    score = mean_squared_error(val_df.Pawpularity, pred)**0.5
    print(f'fold {fold} score : {score}')
    np.save(f'{result_save_dir}val_Bayes_predicts{fold}.npy', pred)
    
    


fold 0 score : 17.5760075136378
fold 1 score : 17.27657823535832
fold 2 score : 17.20668892994361
fold 3 score : 17.15078665592281
fold 4 score : 17.143279261814303
fold 5 score : 17.29698146160812
fold 6 score : 17.755277022466796
fold 7 score : 17.638632344565323
fold 8 score : 17.908811103322712
fold 9 score : 16.919542733698567
